In [5]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.33.2 --progress-bar off
!pip install -qqq langchain==0.0.299 --progress-bar off
!pip install -qqq xformers==0.0.21 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq tokenizers==0.11.1 --progress-bar off


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-pubsublite 1.8.3 requires overrides<7.0.0,>=6.0.1, but you have overrides 7.7.0 which is incompatible.
jupyterlab 4.0.10 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [7]:
import pandas as pd
import json


In [8]:
from huggingface_hub import login
login()

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [9]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"
#MODEL_NAME = "meta-llama/Llama-2-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.001})

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [11]:
from langchain import PromptTemplate

template = """
<s>[INST] <<SYS>>
Act as an Mental Health Clinician.
<</SYS>>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [13]:
from langchain import PromptTemplate

template = """
<s>[INST] <<SYS>>
For a given user post sentence, does it show signs of the symptom. Answer in binary "yes" or "no", for every symptom. The symptoms are as follows:
[Feeling nervous, anxious, or on edge, Not being able to stop or control worrying, Worrying too much about different things, Trouble relaxing, Being so restless that it is hard to sit still, Becoming easily annoyed or irritable, Feeling afraid, as if something awful
might happen].

Give the output as a python list of lists, with the list containing the symptom and the answer, separated by a comma.
{text}:
<</SYS>>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [14]:
text = """
Even during my depression I’ll get these small mini bursts of optimism and motivation thinking, “maybe I can make it?! Maybe I can improve my life and be happy!” And I’ll go to the gym for one day, or run, or try to learn something new, but that’s short lasted. The second I complete the task all I can think is. “What’s the point? I’m probably going to die anyway, so why spend time improving myself?”

I honestly have no will to even live life, all my energy is spent just existing by distracting myself and spending money on stupid shit just to forget about life.

I feel stuck in purgatory. Not courageous enough to kill myself, not motivated enough to make myself happy. I’m stuck. Existing.
"""

In [15]:
print(prompt.format(text=text))


<s>[INST] <<SYS>>
For a given user post sentence, does it show signs of the symptom. Answer in binary "yes" or "no", for every symptom. The symptoms are as follows:
[Feeling nervous, anxious, or on edge, Not being able to stop or control worrying, Worrying too much about different things, Trouble relaxing, Being so restless that it is hard to sit still, Becoming easily annoyed or irritable, Feeling afraid, as if something awful
might happen].

Give the output as a python list of lists, with the list containing the symptom and the answer, separated by a comma.

Even during my depression I’ll get these small mini bursts of optimism and motivation thinking, “maybe I can make it?! Maybe I can improve my life and be happy!” And I’ll go to the gym for one day, or run, or try to learn something new, but that’s short lasted. The second I complete the task all I can think is. “What’s the point? I’m probably going to die anyway, so why spend time improving myself?”

I honestly have no will to e

In [16]:
%%time
result = llm(prompt.format(text=text))

CPU times: user 10.2 s, sys: 439 ms, total: 10.7 s
Wall time: 13.2 s


In [17]:
print(result)

Here is the output as a Python list of lists:

[['Feeling nervous', 'yes'], ['Not being able to stop or control worrying', 'yes'], ['Worrying too much about different things', 'yes'], ['Trouble relaxing', 'yes'], ['Being so restless that it is hard to sit still', 'yes'], ['Becoming easily annoyed or irritable', 'yes'], ['Feeling afraid', 'yes']]

In this output, each symptom is listed in the first column, and the corresponding answer (yes or no) is listed in the second column.


In [19]:
# with open('/kaggle/input/primate/primate_dataset.json', 'r') as file:
#     data = json.load(file)

# Initialize lists for columns
post_titles = []
second_columns = []
i=0
# Process each entry in the JSON data

import pandas as pd
df = pd.read_csv('/kaggle/input/prompt-dataset/prompt_dataset_primate.csv')

for index, row in df.iterrows():
    # Extract post_title and post_text
    print(index)
    post_title = row['Title']
    post_text = row['Text']
    print(f'TITLE: {post_title}')
    #print(f'TEXT: {post_text}')

    # Process post_text using the get_ans function
    second_column_value = llm(prompt.format(text=post_text))

    # Append values to lists
    post_titles.append(post_title)
    second_columns.append(second_column_value)

# Create a DataFrame
df = pd.DataFrame({'Title': post_titles, 'Output': second_columns})

# Write DataFrame to CSV
#df.to_csv('abc.csv', index=False)


0
TITLE: How am I supposed to improve my life when I constantly want to die
1
TITLE: Should I click yes to give the psychiatrist permission to share my information with one of my parents? What are they gonna share?
2
TITLE: What would you do in my position?
3
TITLE: SO says it's depression linked. How to stop being too much?
4
TITLE: Your experience with SSRI's
5
TITLE: Two Faced People - Need Advice
6
TITLE: How can I force myself to do important things when I’m feeling like shit?
7
TITLE: I feel like i'm loosing my best friend and I don't know what to do
8
TITLE: I hate myself so much, I don't deserve to live
9
TITLE: My reward system is fked


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


10
TITLE: My dad just threatened me for crying and going to my room and i would really need some kind words
11
TITLE: Teenager with no hope.
12
TITLE: I'm really depressed and I want to get help but I don't know what to do
13
TITLE: What helped you see some hope for your future, and how can I help my friend see even just a glimmer of hope?
14
TITLE: Placing too much on outside help?
15
TITLE: My grandma just passed away how do I deal with the loss
16
TITLE: It's so painful
17
TITLE: I need help
18
TITLE: Feeling so down and beating myself up for mistakes
19
TITLE: How do I make a female partner who's depressed feel better?
20
TITLE: I feel myself slipping again
21
TITLE: Should I be drunk all the time?
22
TITLE: Self-Improvement help
23
TITLE: Idk how to get out of this funk I’m in
24
TITLE: Am I ungrateful for not being happy?
25
TITLE: Looking for success stories from adults who have been depressed most of their lives
26
TITLE: What is wrong with me? How can I go from content to suic

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


28
TITLE: No matter what I think I feel like I can't stop myself from suicide
29
TITLE: I can’t describe what i felt anymore
30
TITLE: I'm 18F and I don't know if I'm depressed
31
TITLE: What can I do to stop being a weirdo?
32
TITLE: Fell back into binge eating, not taking care of myself, oversleeping.. Spiraling.
33
TITLE: Overthinking is killing me
34
TITLE: Taking antidepressants or not?
35
TITLE: am I suicidal even if I know I won't kill myself?
36
TITLE: My brains are fried...
37
TITLE: Should I tell my therapist I'm suicidal
38
TITLE: "Paying for happiness" - anyone else?
39
TITLE: What can I do…


In [20]:
df.head(10)

,Title,Output
0,How am I supposed to improve my life when I co...,\nHere is the output as a Python list of lists...
1,Should I click yes to give the psychiatrist pe...,\nHere is the output for the given user post s...
2,What would you do in my position?,\nBased on the information provided in the tex...
3,SO says it's depression linked. How to stop be...,Here is the output in a Python list of lists f...
4,Your experience with SSRI's,Here is the output as a Python list of lists:\...
5,Two Faced People - Need Advice,Here is the output of the symptoms present in ...
6,How can I force myself to do important things ...,Here is the output as a Python list of lists:\...
7,I feel like i'm loosing my best friend and I d...,\nHere is the output in a Python list of lists...
8,"I hate myself so much, I don't deserve to live",\nHere is the output of the symptoms present i...
9,My reward system is fked,\nHere are the symptoms present in the given u...


In [21]:
# df.to_csv('llama2_zeroshot_gad_rerun.csv', index=False)
# print('SUCCESS')

SUCCESS
